# OpenStack API

El API de OpenStack permite la gestión de todos los componentes de la infraestructura.

Vamos a realizar las siguientes operaciones como ejemplo de uso:
* autenticación
* creación de red
* creación de router
* creación de máquina virtual

## Variables de entorno
Inicializamos las variables de entorno tomando como partida el archivo de openrc.sh que nos bajamos desde la interfaz de OpenStack:

In [ ]:
import sys
sys.path.append('/usr/lib/python2.7/dist-packages/')

config={}
config["OS_AUTH_URL"]="http://openstack-vcenter:5000/v3"
config["OS_PROJECT_ID"]="9b0989dfbdbd4d048f88720d4ccbf9ac"
config["OS_PROJECT_NAME"]="admin"
config["OS_USER_DOMAIN_NAME"]="default"
config["OS_USERNAME"]="admin"
config["OS_PASSWORD"]="admin"
config["OS_REGION_NAME"]="RegionOne"
config["name"]="prueba"

## Autenticación
Enviamos nuestras credenciales y obtenemos un token en las cabeceras.

In [ ]:
# Peticiones http
import requests
# Tratamiento de json
import json

headers = {}
headers["Content-Type"] = 'application/json'

data = """
{
    "auth": {
        "identity": {
            "methods": [
                "password"
            ],
            "password": {
                "user": {
                    "name": "%s",
                    "password": "%s",
                    "domain": {
                        "name": "%s"
                    }
                }
            }
        },
        "scope": {
            "project": {
                "id": "%s",
                "domain": {
                    "name": "%s"
                }
            }
        }
    }
}
""" % (config["OS_USERNAME"],
       config["OS_PASSWORD"],
       config["OS_USER_DOMAIN_NAME"],
       config["OS_PROJECT_ID"],
       config["OS_USER_DOMAIN_NAME"])
#print data
headers["Content-Type"] = 'application/json'
#
r = requests.post(config["OS_AUTH_URL"] + "/auth/tokens",
                  data=data, headers=headers)
token = json.loads(r.text)
token_id = r.headers["X-Subject-Token"]
print ("token_id: " + str(token_id))
print ("token: " + str(json.dumps(token, indent=4)))




## Creación de red privada
La red de cada PC virtual debe ser independiente y aislada del resto:
* guardamos red con salida pública
* creamos red y subred privada
* creamos puerto en subred privada
* creamos router en subred privada y pública
* asignamos puerto a router


In [ ]:
# Obtenemos el endpoint para el servicio de red
network_url = ""
endpoint_type = "network"
interface_type = "public"
for i in range(len(token["token"]["catalog"])):
    if (token["token"]["catalog"][i]["type"] == endpoint_type):
        for j in range(len(token["token"]["catalog"][i]["endpoints"])):
            if (token["token"]["catalog"][i]["endpoints"][j]["interface"] == interface_type):
                network_url = token["token"]["catalog"][i]["endpoints"][j]["url"]
print (network_url)

In [ ]:
# Preparamos las cabeceras
headers = {}
headers["Content-Type"] = 'application/json'
headers["X-Auth-Token"] = token_id
print json.dumps(headers, indent=4)

In [33]:
# Obtenemos el listado de redes actual
r = requests.get(network_url + "/v2.0/networks", headers=headers)
networks = json.loads(r.text)
print ((json.dumps(networks, indent=4)))

{
    "networks": [
        {
            "provider:physical_network": "external", 
            "ipv6_address_scope": null, 
            "port_security_enabled": true, 
            "provider:network_type": "flat", 
            "id": "a948b2b9-0029-480c-932f-b08c00b1aaf7", 
            "router:external": true, 
            "availability_zone_hints": [], 
            "availability_zones": [
                "nova"
            ], 
            "ipv4_address_scope": null, 
            "shared": false, 
            "status": "ACTIVE", 
            "subnets": [
                "975faac2-e66f-4f0b-b7e9-deb5627b4cb8"
            ], 
            "description": "", 
            "tags": [], 
            "updated_at": "2016-11-24T16:38:09", 
            "is_default": false, 
            "provider:segmentation_id": null, 
            "name": "public", 
            "admin_state_up": true, 
            "tenant_id": "9b0989dfbdbd4d048f88720d4ccbf9ac", 
            "created_at": "2016-11-24T16:38:09", 
 

In [ ]:

# Obtenemos el network_id de la red de publica
public_network = {}
for network in networks["networks"]:
    if network["router:external"]:
        public_network = network
print ((json.dumps(public_network, indent=4)))